# **Self-Ask with Search Agent:**

Self-Ask with Search is an agent framework that enables a large language model (LLM) to answer complex questions by **asking clarifying sub-questions** iteratively and searching for external information. This process involves **breaking down** a complex query into smaller, manageable parts.<br><br>


**How It Works:**<br>
1. **Decomposition:** The agent splits the original question into sub-questions.
2. **Search:** For each sub-question, the agent interacts with external tools (e.g., search engines or APIs) to retrieve relevant data.
3. **Reasoning:** Combines the gathered information to form a final response.
4. **Iteration:** The agent continues asking and searching until sufficient information is obtained.
<br><br>

**Example:**<br>
**Query:** "Who was the U.S. President when the Eiffel Tower was completed, and how tall is the Eiffel Tower?"<br>

**Step-by-Step:**<br>
1. **First Thought:** "I need to determine when the Eiffel Tower was completed."
  * **Action:** Search for "When was the Eiffel Tower completed?"
  * **Observation:** "The Eiffel Tower was completed in 1889."

2. **Second Thought:** "Now, I need to know who was the U.S. President in 1889."
  * **Action:** Search for "U.S. President in 1889."
  * **Observation:** "Benjamin Harrison was the U.S. President in 1889."

3. **Third Thought:** "Finally, I need to find out how tall the Eiffel Tower is."
  * **Action:** Search for "Height of the Eiffel Tower."
  * **Observation:** "The Eiffel Tower is 330 meters tall."

**Final Response:** <br>
"The Eiffel Tower was completed in 1889, during Benjamin Harrison's presidency, and it is 330 meters tall."

In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws wikipedia youtube-search tavily-python

## **Load LLM:**

In [2]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hi there! How can I help you today?



## **Load Embeddings from HF:**

In [ ]:
%pip install --upgrade tokenizers

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Vector Store:**

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [8]:
# retriever:

retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

## **Implementation of Self-ask with Search Agent:**

### **Example 01: Simple Implementation**

In [22]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [23]:
# initialized the Tool:

tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

In [14]:
# Get the prompt to use - you can modify this!

prompt = hub.pull("hwchase17/self-ask-with-search")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'self-ask-with-search', 'lc_hub_commit_hash': '9ef33d1fc82608bdfaa88b3757306ec4f22511b736cb267b1737616641dac0cc'}, template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 

In [24]:
# Construct the ReAct agent:

agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [27]:
# Create an agent executor by passing in the agent and tools,
# and setting handle_parsing_errors to True:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [28]:
# Execute the Agent Executor:

agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is the reigning men's U.S. Open champion?Novak Djokovic is the reigning men's U.S. Open champion, having won the tournament in 2023 by defeating Daniil Medvedev in the final.Follow up: What is the hometown of Novak Djokovic?Novak Djokovic is from Serbia.Could not parse output: So the final answer is: Serbia

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse output: The error you're getting is because the final answer should be Belgrade, Serbia, not just Serbia.  The question asks for the *hometown*, and while Serbia is his home *country*, Belgrade is his *hometown*.

Here's how the decomposition and answers should look to avoid the parsing error:

**Question:** What is the hometown of the reigning men's U.S. Open champion?
**Are followup questions needed here:** Yes.
**Follow up:** Who is the reigning men's U.S. Open champio

Could not parse output: You are absolutely correct. The issue lies in the discrepancy between the question and the final answer provided.  The question asks for the *hometown*, but the provided final answer, "Serbia", is a *country*.  The correct final answer should be **Belgrade, Serbia**.

The breakdown you provided in your edit is the correct way to handle this question:

**Question:** What is the hometown of the reigning men's U.S. Open champion?
**Are followup questions needed here:** Yes.
**Follow up:** Who is the reigning men's U.S. Open champion?
**Intermediate answer:** Novak Djokovic is the reigning men's U.S. Open champion, having won the tournament in 2023 by defeating Daniil Medvedev in the final.
**Follow up:** What is the hometown of Novak Djokovic?
**Intermediate answer:**  Belgrade, Serbia.
**So the final answer is:** Belgrade, Serbia

This corrected version avoids the parsing error because the final answer now accurately and completely addresses the original question.

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

**Note:** Error occurs becz of llm limit issues.